In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import h5py
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
import wandb

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from research.data.natural_scenes import (
    NaturalScenesDataset,
    StimulusDataset,
    KeyDataset
)
from research.models.fmri_decoders import Decoder

from research.experiments.nsd.nsd_experiment import NSDExperiment
from research.metrics.metrics import (
    cosine_similarity, 
    r2_score,
    pearsonr,
    embedding_distance,
    cosine_distance,
    squared_euclidean_distance,
    contrastive_score,
    two_versus_two,
    smooth_euclidean_distance,
    top_knn_test
)
from pipeline.utils import product, index_unsorted

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
nsd_path = Path('D:\\Datasets\\NSD\\')
nsd = NaturalScenesDataset(nsd_path)

In [7]:
from research.experiments.nsd.nsd_run_decoding import main

model_name = 'ViT-B=32'
stimulus_key = 'embedding'
voxel_selection = 'nc'
threshold = 8
num_voxels = None

voxel_selection_path = 'derivatives/noise-ceiling.hdf5'
if threshold is None:
    voxel_selection_key = f'split-01/sorted_indices'
else:
    voxel_selection_key = f'split-01/value'

loss = 'contrastive'


In [ ]:
# Train a single decoder for each participant

group_name = 'group-22'

seed = 0
for subject_id in range(8):
    subject_name = f'subj0{subject_id + 1}'
    experiment = main(
        nsd_path,
        subject_name,
        model_name,
        stimulus_key,
        voxel_selection_path=voxel_selection_path,
        voxel_selection_key=voxel_selection_key,
        group=group_name,
        permutation_test=False,
        max_iterations=5001,
        loss=loss,
        batch_size=128,
        num_voxels=num_voxels,
        threshold=threshold,
        temperature=0.03,
        seed=seed,
    )

In [ ]:
# Train 50 decoders for each participant


group_name = 'group-22_reruns'

for run_id in range(50):
    seed = run_id + 1
    for subject_id in range(8):
        subject_name = f'subj0{subject_id + 1}'
        experiment = main(
            nsd_path,
            subject_name,
            model_name,
            stimulus_key,
            voxel_selection_path=voxel_selection_path,
            voxel_selection_key=voxel_selection_key,
            group=group_name,
            permutation_test=False,
            max_iterations=5001,
            loss=loss,
            batch_size=128,
            num_voxels=num_voxels,
            threshold=threshold,
            temperature=0.03,
            seed=seed,
            result_key=f'run_{run_id}',
        )

In [ ]:
group_name = 'threshold'
temperature = 0.03

results = {}
for subject_id in range(8):
    subject_name = f'subj0{subject_id + 1}'
    results[subject_name] = {}
    for threshold in (8, 9, 10):
        experiment = main(
            nsd_path,
            subject_name,
            model_name,
            stimulus_key,
            voxel_selection_path=voxel_selection_path,
            voxel_selection_key=voxel_selection_key,
            group=group_name,
            permutation_test=False,
            #permutation_fraction=permutation_fraction,
            max_iterations=1501,
            #loss='contrastive',
            loss=loss,
            batch_size=128,
            num_voxels=None,
            threshold=threshold,
            temperature=temperature,
            result_key='wandb_run_name',
        )

In [9]:
# Train ridge regression baseline

from sklearn.linear_model import RidgeCV, Ridge
from research.models.fmri_decoders import Decoder

model_name = 'ViT-B=32'
stimulus_key = 'embedding'

def fit_ridge(subject_id, f, results):
    gc.collect()
    subject_name = f'subj0{subject_id+1}'
    print(subject_name)
    voxel_selection = 'nc'
    threshold = 8.
    num_voxels = None

    if voxel_selection == 'nc':
        voxel_selection_path = 'derivatives/noise-ceiling.hdf5'
        if threshold is None:
            voxel_selection_key = f'split-01/sorted_indices'
        else:
            voxel_selection_key = f'split-01/value'
    elif voxel_selection == 'fracridge':
        voxel_selection_path = f'derivatives/encoded_betas/{model_name}/fracridge.hdf5'
        if threshold is None:
            voxel_selection_key = f'{stimulus_key}/volume_indices'
        else:
            voxel_selection_key = f'{stimulus_key}/value'

    train_mask, val_mask, test_mask = nsd.get_split(subject_name, 'split-01')

    betas_params = dict(
        subject_name=subject_name,
        voxel_selection_path=voxel_selection_path,
        voxel_selection_key=voxel_selection_key,
        num_voxels=num_voxels,
        threshold=threshold,
        return_volume_indices=True,
        return_tensor_dataset=False,
    )
    betas, betas_indices = nsd.load_betas(**betas_params)
    X_train, X_val, X_test = betas[train_mask], betas[val_mask], betas[test_mask]
    print(f'{X_train.shape=}')

    stimulus_params = dict(
        subject_name=subject_name,
        stimulus_path=f'derivatives/stimulus_embeddings/{model_name}.hdf5',
        stimulus_key=stimulus_key,
        delay_loading=False,
        return_tensor_dataset=False,
        return_stimulus_ids=True,
    )
    stimulus, stimulus_ids = nsd.load_stimulus(**stimulus_params)
    stimulus = stimulus.astype(np.float32)
    Y_train, Y_val, Y_test = stimulus[train_mask], stimulus[val_mask], stimulus[test_mask]

    best_model = None
    best_top_1 = 0

    #alpha_grid = (0.1, 1, 10, 100, 1000, 10000, 100000)
    alpha_grid = (1000, 10000)
    alpha_results = results[subject_name] = []
    for alpha in alpha_grid:
        gc.collect()
        model = Ridge(alpha=alpha)
        model.fit(X_train, Y_train)
        Y_val_pred = model.predict(X_val)

        metric = 'cosine'
        top_k_values = [1]

        unique_stimulus_ids, unique_index, unique_inverse = np.unique(stimulus_ids[val_mask], return_index=True, return_inverse=True)
        top_knn_accuracy = top_knn_test(Y_val[unique_index], Y_val_pred, unique_inverse, k=top_k_values, metric=metric)
        print(alpha, top_knn_accuracy)
        alpha_results.append(top_knn_accuracy[0])

        if top_knn_accuracy[0] > best_top_1:
            best_alpha = alpha
            best_top_1 = top_knn_accuracy[0]
            best_model = model

    Y_val_pred = best_model.predict(X_val)
    Y_test_pred = best_model.predict(X_test)

    group = f.create_group(f'{subject_name}/{stimulus_key}')
    group.attrs['alpha'] = best_alpha
    
    group[f'model/weight'] = model.coef_
    group[f'model/bias'] = model.intercept_
    group[f'volume_indices'] = betas_indices
    group[f'val/Y_pred'] = Y_val_pred
    group[f'val/stimulus_ids'] = stimulus_ids[val_mask]
    group[f'test/Y_pred'] = Y_test_pred
    group[f'test/stimulus_ids'] = stimulus_ids[test_mask]

out_path = nsd_path / f'derivatives/decoded_features/{model_name}'
out_path.mkdir(exist_ok=True, parents=True)

with h5py.File(f'{out_path}/ridge-6.hdf5', 'a') as f:
    results = {}
    for subject_id in range(8):
        fit_ridge(subject_id, f, results)
    

subj01
X_train.shape=(21750, 17883)
1000 [0.07133333333333333]
10000 [0.086]
subj02


KeyboardInterrupt: 

In [ ]:
alpha_grid = (1000, 10000)
x = np.arange(len(alpha_grid))
plt.xlabel('alpha')
plt.ylabel('top 1 accuracy')
plt.title('top 1 accuracy vs ridge regression accuracy')
for subject_name, top_1 in results.items():
    plt.plot(x, np.array(top_1) * 100, label=subject_name)
    plt.xticks(x, alpha_grid)
plt.legend()
plt.show()

In [ ]:
results

In [ ]:
results

In [ ]:
with h5py.File(nsd_path / f'derivatives/decoded_features/{model_name}/ridge-1.hdf5', 'w') as f:
    group = f.create_group(f'{subject_name}/{stimulus_key}')
    group[f'model/weight'] = model.coef_
    group[f'model/bias'] = model.intercept_
    group[f'volume_indices'] = betas_indices
    group[f'val/Y_pred'] = Y_val_pred
    group[f'val/stimulus_ids'] = stimulus_ids[val_mask]
    group[f'test/Y_pred'] = Y_test_pred
    group[f'test/stimulus_ids'] = stimulus_ids[test_mask]

In [ ]:
model.intercept_

In [ ]:
gc.collect()

In [ ]:
metric = 'cosine'
top_k_values = [1, 5, 10, 50, 100, 500]

unique_stimulus_ids, unique_index, unique_inverse = np.unique(stimulus_ids[val_mask], return_index=True, return_inverse=True)
top_knn_accuracy = top_knn_test(Y_val[unique_index], Y_val_pred, unique_inverse, k=top_k_values, metric=metric)

In [ ]:
top_knn_accuracy

In [ ]:
wandb_logging = False,

run_models = [
    ('ViT-B=32', 'embedding'),
    #('ViT-B=32', 'transformer.resblocks.3'),
    #('bigbigan-resnet50', 'z_mean'),
    #('DPT_Large', 'scratch.refinenet4'),
]

run_models += [('ViT-B=32', f'transformer.resblocks.{i}') for i in range(12)]

#subjects = nsd.subjects.keys()
#subjects = [f'subj0{i}' for i in range(1, 9)]
#subjects = [f'subj06' for i in range(1, 9)]

for model_name, stimulus_key in run_models:
    for subject_name in subjects:
        def run():
            notes = None

            experiment_params = dict(
                batch_size=128 if stimulus_key == 'embedding' else 64,
                distance_metric='cosine' if stimulus_key == 'embedding' else 'euclidean',
                group='group-4',
                max_iterations = 10001,
                evaluation_interval = 2500,
                channels_last=False, # (model_name == 'ViT-B=32' and stimulus_key != 'embedding'),
                wandb_logging=True,
            )

            betas_params = dict(
                subject_name=subject_name,
                voxel_selection_path='derivatives/voxel-selection.hdf5',
                voxel_selection_key='nc/sorted_indices_flat',
                num_voxels=2500,
                return_volume_indices=True
            )
            betas, betas_indices = nsd.load_betas(**betas_params)

            stimulus_params = dict(
                subject_name=subject_name,
                #stimulus_path='nsddata_stimuli/stimuli/nsd/nsd_stimuli.hdf5',
                #stimulus_key='imgBrick',
                stimulus_path=f'derivatives/stimulus_embeddings/{model_name}.hdf5',
                stimulus_key=stimulus_key,
                delay_loading=True
            )
            stimulus = nsd.load_stimulus(**stimulus_params)

            dataset = KeyDataset({'betas': betas, 'stimulus': stimulus})
            train_dataset, val_dataset, test_dataset = nsd.apply_subject_split(dataset, subject_name, 'split-01')

            config = {'model_name': model_name, **betas_params, **stimulus_params}
            experiment = run_experiment(
                train_dataset,
                val_dataset,
                config=config,
                **experiment_params,
            )


            with torch.no_grad():
                _, Y_val_pred, Y_val_ids = experiment.run_all(val_dataset)
                _, Y_test_pred, Y_test_ids = experiment.run_all(test_dataset)
                _ = None


            def require_dataset(group, key, tensor):
                if key in group:
                    group[key][:] = tensor
                else:
                    group[key] = tensor
            results_path = nsd_path / 'derivatives/decoded_features'

            key_name = wandb.run.group if wandb.run.group else wandb.run.name
            save_file_path = results_path / wandb.config['model_name'] / f'{key_name}.hdf5'
            save_file_path.parent.mkdir(exist_ok=True, parents=True)

            h5_key = (wandb.config['subject_name'], wandb.config['stimulus_key'])

            attributes = dict(wandb.config)
            attributes['wandb_run_name'] = wandb.run.name
            attributes['wandb_run_url'] = wandb.run.url
            attributes['wandb_group'] = wandb.run.group
            attributes['wandb_notes'] = wandb.run.notes

            with h5py.File(save_file_path, 'a') as f:
                key = '/'.join(h5_key)
                group = f.require_group(key)
                for k, v in attributes.items():
                    group.attrs[k] = v
                group.attrs['iteration'] = experiment.iteration
                require_dataset(group, 'volume_indices', betas_indices)
                require_dataset(group, 'test/Y_pred', Y_test_pred.detach().cpu())
                require_dataset(group, 'test/stimulus_ids', Y_test_ids)
                require_dataset(group, 'val/Y_pred', Y_val_pred.detach().cpu())
                require_dataset(group, 'val/stimulus_ids', Y_val_ids)

                model_group = group.require_group('model')
                for param_name, weights in experiment.model.state_dict().items():
                    weights = weights.cpu()
                    require_dataset(model_group, param_name, weights)

            experiment = None
            wandb.finish()
        run()
        torch.cuda.empty_cache()
        gc.collect()


In [ ]:
results_path = nsd_path / 'derivatives/decoded_features'

group_name = 'group-5'
fold = 'val'

eval_models = [
    ('group-10', 'ViT-B=32', 'embedding'),
    #('group-3', 'bigbigan-resnet50', 'z_mean'),
    #('DPT_Large', 'scratch.refinenet4'),
    #('group-5', 'clip-vit-large-patch14-text', 'embedding_unpooled')
]
#eval_models += [('ViT-B=32', f'transformer.resblocks.{i}') for i in range(12)]

subjects = [f'subj0{i}' for i in range(1, 9)]
top_k_values = [1, 5, 10, 50, 100, 500]

results = {}
for group_name, model_name, stimulus_key in eval_models:
    results[(model_name, stimulus_key)] = stimulus_results = {}
    
    stimulus_file = h5py.File(nsd_path / f'derivatives/stimulus_embeddings/{model_name}.hdf5', 'r')
    stimulus = stimulus_file[stimulus_key]
    result_file = h5py.File(nsd_path / f'derivatives/decoded_features/{model_name}/{group_name}.hdf5', 'r')
    
    for subject_name in subjects:
        stimulus_results[subject_name] = subject_results = {}
        print(model_name, stimulus_key, subject_name)
        result = result_file[subject_name][stimulus_key][fold]
        stimulus_ids = result['stimulus_ids'][:]
        
        print('load')
        Y_pred = result['Y_pred'][:]
        Y = index_unsorted(stimulus, stimulus_ids)
        N = Y.shape[0]
        Y_pred = Y_pred.reshape(N, -1)
        Y = Y.reshape(N, -1)
        
        print('knn')
        metric = 'cosine' if stimulus_key == 'embedding' else 'euclidean' 
        
        unique_stimulus_ids, unique_index, unique_inverse = np.unique(stimulus_ids, return_index=True, return_inverse=True)
        top_knn_accuracy = top_knn_test(Y[unique_index], Y_pred, unique_inverse, k=top_k_values, metric=metric)
        subject_results['top_knn_accuracy'] = top_knn_accuracy
        print(subject_results)
        

In [ ]:
N = 1000
chance_accuracy = [k / N for k in top_k_values]

model_name = 'ViT-B=32'
stimulus_key = 'embedding'

plt.figure(figsize=(12, 8))
plt.xticks(ticks=range(len(top_k_values)), labels=top_k_values)
plt.title(f'Top knn accuracy (n={N})\n{model_name=}, {stimulus_key=}')
plt.xlabel('k')
plt.ylabel('accuracy')
plt.plot(range(len(top_k_values)), chance_accuracy, label='chance (k/n)', color='gray')
for subject, subject_results in results[(model_name, stimulus_key)].items():
    top_knn_accuracy = subject_results['top_knn_accuracy']
    plt.plot(range(len(top_k_values)), top_knn_accuracy, label=subject)

plt.grid()
plt.legend()
plt.show()

In [ ]:
N = 1000
chance_accuracy = [k / N for k in top_k_values]

model_name = 'clip-vit-large-patch14-text'
stimulus_key = 'embedding_unpooled'

plt.figure(figsize=(12, 8))
plt.xticks(ticks=range(len(top_k_values)), labels=top_k_values)
plt.title(f'Top knn accuracy (n={N})\n{model_name=}, {stimulus_key=}')
plt.xlabel('k')
plt.ylabel('accuracy')
plt.plot(range(len(top_k_values)), chance_accuracy, label='chance (k/n)', color='gray')
for subject, subject_results in results[(model_name, stimulus_key)].items():
    top_knn_accuracy = subject_results['top_knn_accuracy']
    plt.plot(range(len(top_k_values)), top_knn_accuracy, label=subject)

plt.legend()
plt.show()

In [ ]:
unique_stimulus_ids, unique_indices = np.unique(stimulus_ids, return_index=True)

np.stack([Y_pred[i == stimulus_ids].mean(axis=0) for i in unique_stimulus_ids]).shape

In [ ]:
Y_pred.shape